# Creazione del file PyCalcolAr (file excel CalcolAr automatizzato in codice Python)

### Importare le librerie utili per la creazione del codice 

In [77]:
import pandas as pd
import numpy as np
import math 

from IPython.display import display

### Importare il file triplo36 (file di calibrazione dell'aria):                                
#### per importare il file definire il percorso (file_path, dove trovare il file sul computer)                              

In [83]:
delay = 246

# definire la lista con gli indici delle colonne 
columns_names=['40F', 'err 40F', '38IC0', 'err 38IC0', '36IC1', 'err 36IC1', '36IC0', 'err 36IC0','36F', 'err 36F', 'gain IC0/IC1', 'err gain IC0/IC1', 'gain F/IC1', 'err gain F/IC1' , 'gain F/IC0', 'err gain F/IC0', '40F/36IC1', 'err 40F/36IC1 ', '40F/36F', 'err 40F/36F', '40F/36IC0', 'err 40F/36IC0' , '38IC0/36IC0', 'err 38IC0/36IC0', 'Run', 'Path']

# trasformare le colonne in dati numerici (ad eccezione delle colonne 'Run' e 'Path')
columns_numeric=['40F', 'err 40F', '38IC0', 'err 38IC0', '36IC1', 'err 36IC1', '36IC0', 'err 36IC0','36F', 'err 36F', 'gain IC0/IC1', 'err gain IC0/IC1', 'gain F/IC1', 'err gain F/IC1' , 'gain F/IC0', 'err gain F/IC0', '40F/36IC1', 'err 40F/36IC1 ', '40F/36F', 'err 40F/36F', '40F/36IC0', 'err 40F/36IC0' , '38IC0/36IC0', 'err 38IC0/36IC0']

# definire il percorso del file
file_path = "../test_data/Triplo36.txt"

# importare il file
# utilizzare due caratteri separatori (sep = '\t|,') '\t' = tab, ',' = virgola
airpipette_data = pd.read_csv(file_path, header = None, names = columns_names, sep = '\t|,', engine = 'python') 

In [84]:
# eliminare i caratteri "{}" dalle colonne relative all'errore
airpipette_data = airpipette_data.replace(["{", "}"], ["", ""], regex=True)

# convertire tutte le colonne del dataframe a numeric (float64)
for i in columns_numeric:
    airpipette_data[i] = pd.to_numeric(airpipette_data[i])
    
# definire un dataframe con le "colonne utili" (foglio airpipette_data)
airpipette_data = airpipette_data[['40F', 'err 40F', '38IC0', 'err 38IC0', '36IC1', 'err 36IC1', '36IC0', 'err 36IC0','36F', 'err 36F', 'Run', 'Path']]

# stampare il dataframe 'airpipette_data'
print("Air pipette initial imported data:")
display(airpipette_data)     


Air pipette initial imported data:


,40F,err 40F,38IC0,err 38IC0,36IC1,err 36IC1,36IC0,err 36IC0,36F,err 36F,Run,Path
0,0.73900,0.00400,49900.0,58.0,72000.0,230.0,259000.0,160.0,144000.0,1300.0,'Sample Name' run on 06 October 2015 at 10:33,Raw data stored in - C:\Nu Noble\Results\Data...
1,1.22000,0.00041,49700.0,60.0,75600.0,200.0,258000.0,160.0,251000.0,870.0,'Sample Name' run on 06 October 2015 at 10:52,Raw data stored in - C:\Nu Noble\Results\Data...
2,1.22000,0.00051,49800.0,51.0,76800.0,190.0,259000.0,150.0,251000.0,910.0,'Sample Name' run on 06 October 2015 at 11:23,Raw data stored in - C:\Nu Noble\Results\Data...
3,0.75800,0.00400,49700.0,55.0,66600.0,230.0,258000.0,140.0,144000.0,1100.0,'Sample Name' run on 08 October 2015 at 10:13,Raw data stored in - C:\Nu Noble\Results\Data...
4,1.09000,0.00036,43300.0,63.0,60800.0,170.0,224000.0,130.0,220000.0,1400.0,'Sample Name' run on 08 October 2015 at 10:27,Raw data stored in - C:\Nu Noble\Results\Data...
...,...,...,...,...,...,...,...,...,...,...,...,...
829,1.65538,0.00053,55318.2,79.0,307546.0,250.0,298052.0,240.0,347513.0,1100.0,'Sample Name' run on 23 June 2020 at 11:02,Raw data stored in - C:\Nu Noble\Results\Data...
830,1.65331,0.00043,55167.7,90.0,305585.0,220.0,297394.0,300.0,349141.0,1300.0,'Sample Name' run on 24 June 2020 at 09:25,Raw data stored in - C:\Nu Noble\Results\Data...
831,1.65039,0.00049,55606.8,82.0,306701.0,200.0,299474.0,260.0,347983.0,1300.0,'Sample Name' run on 24 June 2020 at 09:50,Raw data stored in - C:\Nu Noble\Results\Data...
832,1.64847,0.00056,55592.8,63.0,306848.0,260.0,299066.0,230.0,346598.0,1400.0,'Sample Name' run on 24 June 2020 at 10:08,Raw data stored in - C:\Nu Noble\Results\Data...


### Il dataframe deve essere filtrato per data 
#### (per poter scegliere la riga di calibrazione d'interesse)

In [85]:
# dividere la colonna 'Run' in due colonne: nome del run e data/ora
run_split = airpipette_data['Run'].str.split(" run on ")

# formattare la colonna con il nome del run (del campione) in una serie pandas e associarle un nome 
run_name = run_split.str[0]
run_name = run_name.replace(["'"], [""], regex=True)
run_name.name = 'Run_Name'

# formattare la colonna con la data e l'ora in una serie pandas e associarle un nome, convertire il dato in datetime64
dataora = run_split.str[1]
dataora.name = 'Date_Time'
dataora = pd.to_datetime(dataora)

# concatenare le due nuove colonne all'inizio del dataframe airpipette_data
airpipette_data = pd.concat([run_name, dataora, airpipette_data], axis = 1)

# eliminare la colonna 'Run' (non più utilizzata)
airpipette_data.drop('Run',axis=1, inplace=True)
display(airpipette_data)

,Run_Name,Date_Time,40F,err 40F,38IC0,err 38IC0,36IC1,err 36IC1,36IC0,err 36IC0,36F,err 36F,Path
0,Sample Name,2015-10-06 10:33:00,0.73900,0.00400,49900.0,58.0,72000.0,230.0,259000.0,160.0,144000.0,1300.0,Raw data stored in - C:\Nu Noble\Results\Data...
1,Sample Name,2015-10-06 10:52:00,1.22000,0.00041,49700.0,60.0,75600.0,200.0,258000.0,160.0,251000.0,870.0,Raw data stored in - C:\Nu Noble\Results\Data...
2,Sample Name,2015-10-06 11:23:00,1.22000,0.00051,49800.0,51.0,76800.0,190.0,259000.0,150.0,251000.0,910.0,Raw data stored in - C:\Nu Noble\Results\Data...
3,Sample Name,2015-10-08 10:13:00,0.75800,0.00400,49700.0,55.0,66600.0,230.0,258000.0,140.0,144000.0,1100.0,Raw data stored in - C:\Nu Noble\Results\Data...
4,Sample Name,2015-10-08 10:27:00,1.09000,0.00036,43300.0,63.0,60800.0,170.0,224000.0,130.0,220000.0,1400.0,Raw data stored in - C:\Nu Noble\Results\Data...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
829,Sample Name,2020-06-23 11:02:00,1.65538,0.00053,55318.2,79.0,307546.0,250.0,298052.0,240.0,347513.0,1100.0,Raw data stored in - C:\Nu Noble\Results\Data...
830,Sample Name,2020-06-24 09:25:00,1.65331,0.00043,55167.7,90.0,305585.0,220.0,297394.0,300.0,349141.0,1300.0,Raw data stored in - C:\Nu Noble\Results\Data...
831,Sample Name,2020-06-24 09:50:00,1.65039,0.00049,55606.8,82.0,306701.0,200.0,299474.0,260.0,347983.0,1300.0,Raw data stored in - C:\Nu Noble\Results\Data...
832,Sample Name,2020-06-24 10:08:00,1.64847,0.00056,55592.8,63.0,306848.0,260.0,299066.0,230.0,346598.0,1400.0,Raw data stored in - C:\Nu Noble\Results\Data...


### Utente: scegliere la data d'interesse

In [86]:
# filtrare il dataframe per data, prendendo una sola riga 
import ipywidgets as widgets

data_w = widgets.Dropdown(options=airpipette_data['Date_Time'].to_list(),
                 value=airpipette_data['Date_Time'].to_list()[-1],
                 description='Date_Time of calibration:',
                 disabled=False)
display(data_w)

Dropdown(description='Date_Time of calibration:', index=833, options=(Timestamp('2015-10-06 10:33:00'), Timest…

### Utente: selezionare la data d'interesse nell'elenco qui sopra, prima di eseguire (runnare) la cella successiva  

In [16]:
print(data_w.value)
airpipette_data_filtered = airpipette_data[airpipette_data['Date_Time'] == data_w.value]

calibration_data = airpipette_data_filtered 
display(calibration_data)


2020-06-29 09:41:00


,Run_Name,Date_Time,40F,err 40F,38IC0,err 38IC0,36IC1,err 36IC1,36IC0,err 36IC0,36F,err 36F,Path
833,Sample Name,2020-06-29 09:41:00,1.65661,0.00047,54386.0,85.0,305470.0,280.0,293720.0,330.0,346964.0,1200.0,Raw data stored in - C:\Nu Noble\Results\Data...


### importare i file Run4 e Run22 
(con i medesimi passaggi svolti per triplo36)

In [17]:
# definire la lista con gli indici delle colonne 
column_names_run4 = ['40Ar F', 'err40Ar F', '38Ar IC0', 'err38Ar IC0', '36Ar IC1', 'err36Ar IC1','38Ar F', 'err38Ar F','36Ar IC0', 'err36Ar IC0', '39Ar F', 'err39Ar F', '37Ar IC0', 'err37Ar IC0' , '35Cl IC1', 'err35Cl IC1', '39Ar IC0',  'err39Ar IC0',  '37Ar IC1', 'err37Ar IC1', 'gainF/IC0', 'err gainF/IC0' , 'gainIC0/IC1', 'err gainIC0/IC1', '40F/36IC1',  'err40F/36IC1',  '40F/36IC0' ,'err40F/36IC0', 'Run', 'Path']
column_names_run22 = ['40Ar F', 'err40Ar F', '38Ar IC0', 'err38Ar IC0', '36Ar IC1', 'err36Ar IC1','38Ar F', 'err38Ar F','36Ar IC0', 'err36Ar IC0', '39Ar F', 'err39Ar F', '37Ar IC0', 'err37Ar IC0' , '35Cl IC1', 'err35Cl IC1', 'gainIC0/IC1', 'err gainIC0/IC1' , '40F/36IC1',  'err40F/36IC1',  '40F/36IC0' ,'err40F/36IC0', 'Run', 'Path']

# definire il percorso dei file
file_path_4 = "../test_data/Run4.txt"
file_path_22 = "../test_data/Run22.txt"

# importare i file run
df_data_4 = pd.read_csv(file_path_4, header = None, index_col= False, names = column_names_run4, sep = '\t|,', engine = 'python')
df_data_22 = pd.read_csv(file_path_22, header = None, index_col= False, names = column_names_run22, sep = '\t|,', engine = 'python')

In [18]:
# eliminare i caratteri "{}" dalle colonne
df_data_4 = df_data_4.replace(["{", "}"], ["", ""], regex=True)
df_data_22 = df_data_22.replace(["{", "}"], ["", ""], regex=True)

In [19]:
# definire nome delle colonne che contengono dati numerici
colnames_numeric_4 = ['40Ar F', 'err40Ar F', '38Ar IC0', 'err38Ar IC0', '36Ar IC1', 'err36Ar IC1','38Ar F', 'err38Ar F','36Ar IC0', 'err36Ar IC0', '39Ar F', 'err39Ar F', '37Ar IC0', 'err37Ar IC0' , '35Cl IC1', 'err35Cl IC1', '39Ar IC0',  'err39Ar IC0',  '37Ar IC1', 'err37Ar IC1', 'gainF/IC0', 'err gainF/IC0' , 'gainIC0/IC1', 'err gainIC0/IC1', '40F/36IC1',  'err40F/36IC1',  '40F/36IC0' ,'err40F/36IC0']   
colnames_numeric_22 = ['40Ar F', 'err40Ar F', '38Ar IC0', 'err38Ar IC0', '36Ar IC1', 'err36Ar IC1','38Ar F', 'err38Ar F','36Ar IC0', 'err36Ar IC0', '39Ar F', 'err39Ar F', '37Ar IC0', 'err37Ar IC0' , '35Cl IC1', 'err35Cl IC1', 'gainIC0/IC1', 'err gainIC0/IC1' , '40F/36IC1',  'err40F/36IC1',  '40F/36IC0' ,'err40F/36IC0']

# convertire tutte le colonne del dataframe df_data_4 a numeric (float64)
for i in colnames_numeric_4:
    df_data_4[i] = pd.to_numeric(df_data_4[i])

# convertire tutte le colonne del dataframe df_data_22 a numeric (float64)
for i in colnames_numeric_22:
    df_data_22[i] = pd.to_numeric(df_data_22[i])

### Dopo aver importato i due file come dataframe, verranno uniti a formare un unico dataframe 

In [20]:
df_data = pd.concat([df_data_4, df_data_22], axis=0)
display(df_data)

,40Ar F,err40Ar F,38Ar IC0,err38Ar IC0,36Ar IC1,err36Ar IC1,38Ar F,err38Ar F,36Ar IC0,err36Ar IC0,...,gainF/IC0,err gainF/IC0,gainIC0/IC1,err gainIC0/IC1,40F/36IC1,err40F/36IC1,40F/36IC0,err40F/36IC0,Run,Path
0,0.847000,0.000370,0.000493,3.700000e-07,0.002460,3.900000e-06,0.000047,0.000015,0.002610,1.300000e-06,...,0.396,0.064,1.060000,0.0018,345.000,0.57,325.000,0.21,'fishcanyon bt stp1' run on 08 April 2015 at 1...,Raw data stored in - C:\Nu Noble\Results\Data...
1,2.300000,0.000870,0.001540,8.500000e-07,0.006340,9.200000e-06,0.000107,0.000013,0.006510,2.500000e-06,...,2.670,0.018,1.030000,0.0015,363.000,0.55,354.000,0.19,'fishcanyon bt stp2' run on 08 April 2015 at 1...,Raw data stored in - C:\Nu Noble\Results\Data...
2,1.080000,0.000350,0.000750,5.000000e-07,0.002850,5.400000e-06,0.000037,0.000012,0.003050,1.400000e-06,...,4.340,0.029,1.070000,0.0021,378.000,0.72,354.000,0.20,'Sample Name' run on 08 April 2015 at 17:28,Raw data stored in - C:\Nu Noble\Results\Data...
3,0.089300,0.000040,0.000059,1.300000e-07,0.000215,2.700000e-07,0.000009,0.000012,0.000259,2.900000e-07,...,2.160,0.260,1.210000,0.0020,416.000,0.55,345.000,0.41,'Sample Name' run on 09 April 2015 at 11:39,Raw data stored in - C:\Nu Noble\Results\Data...
4,0.578000,0.000270,0.000329,3.000000e-07,0.001530,2.400000e-06,0.000050,0.000011,0.001700,8.900000e-07,...,48700.000,48000.000,1.110000,0.0018,378.000,0.61,340.000,0.24,'Sample Name' run on 09 April 2015 at 12:11,Raw data stored in - C:\Nu Noble\Results\Data...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,0.426141,0.000120,0.003709,2.600000e-06,0.000883,7.200000e-07,0.004290,0.000017,0.000881,8.000000e-07,...,NaN,NaN,0.997925,0.0012,482.535,0.42,483.538,0.46,'BAL-T12 LG ST2' run on 23 June 2020 at 15:31,Raw data stored in - C:\Nu Noble\Results\Data...
1334,0.224427,0.000066,0.003784,2.800000e-06,0.000268,4.000000e-07,0.004378,0.000017,0.000268,4.100000e-07,...,NaN,NaN,1.003020,0.0022,838.663,1.30,836.139,1.30,'BAL-T12 LG ST3' run on 23 June 2020 at 15:57,Raw data stored in - C:\Nu Noble\Results\Data...
1335,0.421204,0.000130,0.004853,3.600000e-06,0.000743,7.200000e-07,0.005579,0.000016,0.000749,6.800000e-07,...,NaN,NaN,1.008230,0.0013,567.014,0.58,562.386,0.54,'BAL-T12 LG ST6' run on 23 June 2020 at 17:08,Raw data stored in - C:\Nu Noble\Results\Data...
1336,0.585912,0.000160,0.011360,1.100000e-05,0.000572,6.300000e-07,0.012936,0.000016,0.000581,6.800000e-07,...,NaN,NaN,1.015770,0.0016,1024.300,1.20,1008.400,1.20,'BAL-T12 ST7' run on 23 June 2020 at 17:26,Raw data stored in - C:\Nu Noble\Results\Data...


### il dataframe deve essere filtrato per data 
(come fatto per triplo36)

In [21]:
# dividere la colonna 'Run' in due colonne: nome del run e data/ora
run_split = df_data['Run'].str.split(" run on ")

# formattare la colonna con il nome del run (del campione) in una serie pandas e associarle un nome 
run_name = run_split.str[0]
run_name = run_name.replace(["'"], [""], regex=True)
run_name.name = 'Run_Name'

# formattare la colonna con la data e l'ora in una serie pandas e associarle un nome, convertire il dato in datetime64
dataora = run_split.str[1]
dataora.name = 'Date_Time'
dataora = pd.to_datetime(dataora)

# concatenare le due nuove colonne all'inizio del dataframe df_data
df_data = pd.concat([run_name, dataora, df_data], axis = 1)

# eliminare la colonna 'Run' (non più utilizzata)
df_data.drop('Run',axis=1, inplace=True)
display(df_data)

,Run_Name,Date_Time,40Ar F,err40Ar F,38Ar IC0,err38Ar IC0,36Ar IC1,err36Ar IC1,38Ar F,err38Ar F,...,err37Ar IC1,gainF/IC0,err gainF/IC0,gainIC0/IC1,err gainIC0/IC1,40F/36IC1,err40F/36IC1,40F/36IC0,err40F/36IC0,Path
0,fishcanyon bt stp1,2015-04-08 16:03:00,0.847000,0.000370,0.000493,3.700000e-07,0.002460,3.900000e-06,0.000047,0.000015,...,6.600000e-09,0.396,0.064,1.060000,0.0018,345.000,0.57,325.000,0.21,Raw data stored in - C:\Nu Noble\Results\Data...
1,fishcanyon bt stp2,2015-04-08 16:33:00,2.300000,0.000870,0.001540,8.500000e-07,0.006340,9.200000e-06,0.000107,0.000013,...,1.000000e-08,2.670,0.018,1.030000,0.0015,363.000,0.55,354.000,0.19,Raw data stored in - C:\Nu Noble\Results\Data...
2,Sample Name,2015-04-08 17:28:00,1.080000,0.000350,0.000750,5.000000e-07,0.002850,5.400000e-06,0.000037,0.000012,...,8.200000e-09,4.340,0.029,1.070000,0.0021,378.000,0.72,354.000,0.20,Raw data stored in - C:\Nu Noble\Results\Data...
3,Sample Name,2015-04-09 11:39:00,0.089300,0.000040,0.000059,1.300000e-07,0.000215,2.700000e-07,0.000009,0.000012,...,4.400000e-08,2.160,0.260,1.210000,0.0020,416.000,0.55,345.000,0.41,Raw data stored in - C:\Nu Noble\Results\Data...
4,Sample Name,2015-04-09 12:11:00,0.578000,0.000270,0.000329,3.000000e-07,0.001530,2.400000e-06,0.000050,0.000011,...,6.400000e-09,48700.000,48000.000,1.110000,0.0018,378.000,0.61,340.000,0.24,Raw data stored in - C:\Nu Noble\Results\Data...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,BAL-T12 LG ST2,2020-06-23 15:31:00,0.426141,0.000120,0.003709,2.600000e-06,0.000883,7.200000e-07,0.004290,0.000017,...,NaN,NaN,NaN,0.997925,0.0012,482.535,0.42,483.538,0.46,Raw data stored in - C:\Nu Noble\Results\Data...
1334,BAL-T12 LG ST3,2020-06-23 15:57:00,0.224427,0.000066,0.003784,2.800000e-06,0.000268,4.000000e-07,0.004378,0.000017,...,NaN,NaN,NaN,1.003020,0.0022,838.663,1.30,836.139,1.30,Raw data stored in - C:\Nu Noble\Results\Data...
1335,BAL-T12 LG ST6,2020-06-23 17:08:00,0.421204,0.000130,0.004853,3.600000e-06,0.000743,7.200000e-07,0.005579,0.000016,...,NaN,NaN,NaN,1.008230,0.0013,567.014,0.58,562.386,0.54,Raw data stored in - C:\Nu Noble\Results\Data...
1336,BAL-T12 ST7,2020-06-23 17:26:00,0.585912,0.000160,0.011360,1.100000e-05,0.000572,6.300000e-07,0.012936,0.000016,...,NaN,NaN,NaN,1.015770,0.0016,1024.300,1.20,1008.400,1.20,Raw data stored in - C:\Nu Noble\Results\Data...


### Filtrare set di dati per data
### Utente: selezionare la data d'interesse modificando filter_data_min e filter_data_max

In [47]:
type(df_data['Date_Time'])
# filtrare il dataframe per data 
filter_data_min = pd.Timestamp(year=2020, month=6, day=20, hour=0)
filter_data_max = pd.Timestamp(year=2020, month=6, day=23, hour=17)

df_data = df_data[(df_data['Date_Time'] > filter_data_min) & \
                                           (df_data['Date_Time'] < filter_data_max)]
df_data.reset_index(drop=True, inplace=True)
display(df_data)

,Run_Name,Date_Time,40Ar F,err40Ar F,38Ar IC0,err38Ar IC0,36Ar IC1,err36Ar IC1,38Ar F,err38Ar F,...,err37Ar IC1,gainF/IC0,err gainF/IC0,gainIC0/IC1,err gainIC0/IC1,40F/36IC1,err40F/36IC1,40F/36IC0,err40F/36IC0,Path
0,MH-T4 ST1,2020-06-22 12:38:00,0.392709,0.000120,0.000921,7.600000e-07,0.001070,8.100000e-07,0.001098,0.000014,...,7.600000e-08,1.20190,0.0025,0.973208,0.00110,367.033,0.30,377.138,0.31,Raw data stored in - C:\Nu Noble\Results\Data...
1,MH-T4 ST5,2020-06-22 14:24:00,0.244726,0.000076,0.001673,1.100000e-06,0.000556,5.500000e-07,0.001935,0.000016,...,7.900000e-08,1.17424,0.0016,0.989598,0.00150,440.520,0.45,445.151,0.52,Raw data stored in - C:\Nu Noble\Results\Data...
2,MH-T4 ST6,2020-06-22 14:46:00,0.216365,0.000074,0.001035,7.500000e-07,0.000551,5.300000e-07,0.001196,0.000017,...,6.600000e-08,1.17538,0.0022,0.990113,0.00140,392.984,0.40,396.909,0.42,Raw data stored in - C:\Nu Noble\Results\Data...
3,MH-T4 ST7,2020-06-22 15:08:00,0.225166,0.000085,0.001009,7.800000e-07,0.000575,5.800000e-07,0.001187,0.000014,...,7.700000e-08,1.16900,0.0020,0.991632,0.00130,391.283,0.42,394.585,0.39,Raw data stored in - C:\Nu Noble\Results\Data...
4,MH-T4 ST8,2020-06-22 15:30:00,0.236172,0.000086,0.001415,1.000000e-06,0.000545,5.100000e-07,0.001629,0.000017,...,8.200000e-08,1.15975,0.0017,0.997047,0.00140,433.712,0.44,434.997,0.47,Raw data stored in - C:\Nu Noble\Results\Data...
5,BAL-T12 SM ST1,2020-06-23 12:02:00,0.277142,0.000093,0.000348,5.200000e-07,0.000792,7.300000e-07,0.000382,0.000017,...,6.600000e-08,1.19585,0.0120,0.972248,0.00130,350.099,0.34,360.092,0.39,Raw data stored in - C:\Nu Noble\Results\Data...
6,BAL-T12 SM ST2,2020-06-23 12:27:00,0.516114,0.000160,0.001671,1.000000e-06,0.001361,8.500000e-07,0.001929,0.000014,...,8.700000e-08,1.16794,0.0015,0.990284,0.00098,379.123,0.26,382.843,0.32,Raw data stored in - C:\Nu Noble\Results\Data...
7,BAL-T12 SM ST3,2020-06-23 12:48:00,0.134762,0.000038,0.001201,8.600000e-07,0.000280,4.200000e-07,0.001408,0.000014,...,8.100000e-08,1.16792,0.0017,0.998740,0.00200,482.110,0.73,482.718,0.69,Raw data stored in - C:\Nu Noble\Results\Data...
8,BAL-T12 AM ST4,2020-06-23 13:15:00,0.177961,0.000059,0.001600,1.000000e-06,0.000371,4.100000e-07,0.001839,0.000014,...,7.900000e-08,1.15714,0.0015,1.002560,0.00160,479.553,0.55,478.329,0.57,Raw data stored in - C:\Nu Noble\Results\Data...
9,BAL-T12 LG ST1,2020-06-23 15:10:00,0.963265,0.000270,0.001579,9.300000e-07,0.002737,1.500000e-06,0.001843,0.000017,...,8.700000e-08,1.16399,0.0023,0.991731,0.00079,351.999,0.22,354.934,0.22,Raw data stored in - C:\Nu Noble\Results\Data...


### Operazioni derivate dal file di calibrazione

per calcolare 1sig_rel (errore relativo) = err / 36Ar (IC0 o IC1 in base al collettore usato)

In [48]:
value_err36IC0 = float(calibration_data['err 36IC0'].values)
value_36IC0 = float(calibration_data['36IC0'].values)
sig_rel_36IC0 = float(value_err36IC0 / value_36IC0)
print ('1sig_rel_36IC0', sig_rel_36IC0)

1sig_rel_36IC0 0.00112351899768487


In [49]:
value_err36IC1 = float(calibration_data['err 36IC1'].values)
value_36IC1 = float(calibration_data['36IC1'].values)
sig_rel_36IC1 = float(value_err36IC1 / value_36IC1)
print ('1sig_rel_36IC1', sig_rel_36IC1)

1sig_rel_36IC1 0.000916620290044849


In [50]:
value_err36F = float(calibration_data['err 36F'].values)
value_36F = float(calibration_data['36F'].values)
sig_rel_36F = float(value_err36F / value_36F)
print ('1sig_rel_36F', sig_rel_36F)

1sig_rel_36F 0.0034585720708776705


In [51]:
# calcolo gain F/IC0
value_36F = float(calibration_data['36F'].values)
value_36IC0 = float(calibration_data['36IC0'].values)
gain_F_IC0 = float(value_36F / value_36IC0)
print ('gain F/IC0', gain_F_IC0)

gain F/IC0 1.1812746833719188


In [52]:
# calcolo l'errore assoluto del gain F/IC0
sig_abs_F_ICO = gain_F_IC0 * (pow(sig_rel_36IC0, 2) + pow(sig_rel_36F, 2))**(1/2)
print ('sig_abs_F/ICO', sig_abs_F_ICO)

sig_abs_F/ICO 0.0042956864573216935


In [53]:
# calcolo gain F/IC1 
value_36F = float(calibration_data['36F'].values)
value_36IC1 = float(calibration_data['36IC1'].values)
gain_F_IC1 = float(value_36F / value_36IC1)
print ('gain F/IC1: ', gain_F_IC1)

gain F/IC1:  1.1358365796968606


In [54]:
# calcolo l'errore assoluto del gain F/IC1
sig_abs_F_IC1 = gain_F_IC1 * (pow(sig_rel_36IC1, 2) + pow(sig_rel_36F, 2))**(1/2)
print ('sig_abs_F/IC1', sig_abs_F_IC1)

sig_abs_F/IC1 0.004063996223501063


In [55]:
# calcolo 36IC0 correzione gain
corr_gain_36IC0 = value_36IC0 * gain_F_IC0
print ('corr_gain_36IC0: ', corr_gain_36IC0) 

corr_gain_36IC0:  346964.0


In [56]:
# calcolo 40/36 correzione
value_40F = float(calibration_data['40F'].values)
B = 62415000 #aggiungere descrizione della constante
corr_40_36 = (value_40F * B) / corr_gain_36IC0
print ('corr_40_36: ', corr_40_36)

corr_40_36:  298.0058828869854


In [57]:
# calcolo l'errore assoluto del 40/36 correzione
sig_abs_40_36 = corr_40_36 * (pow(sig_rel_36F, 2) + pow(calibration_data['err 40F'].values / calibration_data['40F'].values, 2))**(1/2)
print ('sig_abs_40/36', sig_abs_40_36)

sig_abs_40/36 [1.0341368]


In [58]:
# calcolo source frax 
source_frax = float(corr_40_36 / 298.56)
print ('source_frax: ', source_frax)

source_frax:  0.9981440343213605


In [59]:
# calcolo l'errore assoluto di source frax
sig_abs_source_frax = source_frax * (sig_abs_40_36 / corr_40_36)
print ('sig_abs_source_frax', sig_abs_source_frax)

sig_abs_source_frax [0.00346375]


Definisco tutti i parametri delle tabelle (A) e (B) del file excel (non sappiamo ancora da dove vengono presi tutti)

In [97]:
# da modificare a mano 

update_data = pd.Timestamp(year=2021, month=1, day=23)

background_spectrometer_dict = {'Update date': [update_data, np.nan],
                                'Background 40Ar': [0.0020, 0.1],
                                'Background 39Ar': [0.0030, 0.1],
                                'Background 38Ar': [0.0010, 0.1],
                                'Background 37Ar': [0.0040, 0.2],
                                'Background 36Ar': [0.0005, 0.1]}

background_spectrometer_df = pd.DataFrame.from_dict(background_spectrometer_dict, orient='index')
background_spectrometer_df.columns = ['value', 'relative error']

display(background_spectrometer_df)

,value,relative error
Update date,2021-01-23 00:00:00,NaN
Background 40Ar,0.002,0.1
Background 39Ar,0.003,0.1
Background 38Ar,0.001,0.1
Background 37Ar,0.004,0.2
Background 36Ar,0.0005,0.1


In [98]:
irradiations_constants_dict = {'Atmospheric Ratio': [298.56, np.nan],
                                '(36Ar/37Ar) Ca': [0.000255, 0.015],
                                '(38Ar/37Ar) Ca': [0.00027, 0.015],
                                '(39Ar/37Ar) Ca': [0.00067, 0.015],
                                'Lambda Ar37 [1/d]': [0.019748, np.nan],
                                'Lambda Ar40 [1/Ma]': [0.0005543, np.nan],
                                'Interference 40K': [0.033, 0.01],
                                'Coefficient 39Ar for J': [143, np.nan],
                                'Coefficient Ca/K': [1.94, np.nan],
                                'Coefficient Cl/K': [0.178, np.nan]}

irradiations_constants_df = pd.DataFrame.from_dict(irradiations_constants_dict, orient='index')
irradiations_constants_df.columns = ['value', 'relative error']

display(irradiations_constants_df)


,value,relative error
Atmospheric Ratio,298.560000,NaN
(36Ar/37Ar) Ca,0.000255,0.015
(38Ar/37Ar) Ca,0.000270,0.015
(39Ar/37Ar) Ca,0.000670,0.015
Lambda Ar37 [1/d],0.019748,NaN
Lambda Ar40 [1/Ma],0.000554,NaN
Interference 40K,0.033000,0.010
Coefficient 39Ar for J,143.000000,NaN
Coefficient Ca/K,1.940000,NaN
Coefficient Cl/K,0.178000,NaN


In [99]:
sample_weight = 0.00034
J_factor = 0.0011422
J_factor_errors = 0.0005
sensitivity = 0.0000000000337 
gain_F_IC0 = gain_F_IC0 # già calcolato precedentemente 
gain_rel_uncertainty_errors = calibration_data['err 40F'].values / calibration_data['err 36F'].values
gain_rel_uncertainty = gain_rel_uncertainty_errors / gain_F_IC0
measured_40Ar_36Ar_pipettes = calibration_data['40F'].values / calibration_data['36IC1'].values
gain_corrected_40Ar_36Ar_pipettes = measured_40Ar_36Ar_pipettes / gain_F_IC0
gain_corrected_40Ar_36Ar_pipettes_errors = gain_corrected_40Ar_36Ar_pipettes / irradiations_constants_df.loc['Atmospheric Ratio', 'value']
pipette_rel_uncertainty = sig_abs_source_frax  
total_fractionation_uncertainty = sig_abs_source_frax 

irradiations_dict = {'Sample weight [g]': [sample_weight, np.nan],
                     'J factor': [J_factor, J_factor_errors],
                     'Sensitivity (mL/mV)': [sensitivity, np.nan],
                     'Gain F/IC0': [gain_F_IC0, np.nan],
                     'Gain rel uncertainty': [gain_rel_uncertainty[0], gain_rel_uncertainty_errors[0]],
                     'Measured 40Ar/36Ar pipettes': [measured_40Ar_36Ar_pipettes[0], np.nan],
                     'Gain corrected 40Ar/36Ar pipettes': [gain_corrected_40Ar_36Ar_pipettes[0], gain_corrected_40Ar_36Ar_pipettes_errors[0]], 
                     'Pipette rel uncertainty': [pipette_rel_uncertainty[0], np.nan],
                     'Total fractionation uncertainty': [total_fractionation_uncertainty[0], np.nan]} 

irradiations_df = pd.DataFrame.from_dict(irradiations_dict, orient='index')
irradiations_df.columns = ['value', 'relative error']

display(irradiations_df)

,value,relative error
Sample weight [g],3.400000e-04,NaN
J factor,1.142200e-03,5.000000e-04
Sensitivity (mL/mV),3.370000e-11,NaN
Gain F/IC0,1.181275e+00,NaN
Gain rel uncertainty,3.315627e-07,3.916667e-07
Measured 40Ar/36Ar pipettes,5.423151e-06,NaN
Gain corrected 40Ar/36Ar pipettes,4.590932e-06,1.537691e-08
Pipette rel uncertainty,3.463749e-03,NaN
Total fractionation uncertainty,3.463749e-03,NaN


###### Svolgimento operazioni presenti nel fogli excel CalcolAr

Le operazione necessarie verranno aggiunte in un unico dataframe di risultati simile a quello del file excel 


In [100]:
input_data_df = pd.DataFrame()

input_data_df.loc[:, 'Time'] = pd.Series(1 for k in range(0, len(df_data.index)))
input_data_df.loc[:, 'Delay'] = pd.Series(delay for k in range(0, len(df_data.index)))


input_data_df.loc[:, '40Ar'] = df_data.loc[:, '40Ar F'].values * 1000
input_data_df.loc[:, 'err40Ar'] = df_data.loc[:, 'err40Ar F'].values * 1000

input_data_df.loc[:, '39Ar'] = df_data.loc[:, '39Ar F'].values * 1000
input_data_df.loc[:, 'err39Ar'] = df_data.loc[:, 'err39Ar F'].values * 1000

input_data_df.loc[:, '38Ar'] = df_data.loc[:, '40Ar F'].values * 1000
input_data_df.loc[:, 'err38Ar'] = df_data.loc[:, 'err38Ar F'].values * 1000

input_data_df.loc[:, '37Ar'] = df_data.loc[:, '37Ar IC0'].values * df_data.loc[:, 'gainF/IC0'].values * 1000
input_data_df.loc[:, 'err37Ar'] = df_data.loc[:, 'err37Ar IC0'].values * df_data.loc[:, 'gainF/IC0'].values * 1000

input_data_df.loc[:, '36Ar'] = df_data.loc[:, '36Ar IC0'].values * df_data.loc[:, 'gainF/IC0'].values * 1000
input_data_df.loc[:, 'err36Ar'] = df_data.loc[:, 'err36Ar IC0'].values * df_data.loc[:, 'gainF/IC0'].values * 1000

display(input_data_df)

,Time,Delay,40Ar,err40Ar,39Ar,err39Ar,38Ar,err38Ar,37Ar,err37Ar,36Ar,err36Ar
0,1,246,392.709,0.120,6.95503,0.014,392.709,0.014,0.008274,0.000076,1.251526,0.000962
1,1,246,244.726,0.076,16.39790,0.017,244.726,0.016,0.007669,0.000066,0.645550,0.000728
2,1,246,216.365,0.074,9.87657,0.016,216.365,0.017,0.007688,0.000071,0.640730,0.000646
3,1,246,225.166,0.085,10.35230,0.016,225.166,0.014,0.007593,0.000068,0.667077,0.000608
4,1,246,236.172,0.086,15.62530,0.019,236.172,0.017,0.007757,0.000074,0.629661,0.000638
5,1,246,277.142,0.093,1.61851,0.016,277.142,0.017,0.008214,0.000074,0.920375,0.000945
6,1,246,516.114,0.160,14.88850,0.014,516.114,0.014,0.009345,0.000072,1.574512,0.001168
7,1,246,134.762,0.038,12.27530,0.014,134.762,0.014,0.008279,0.000083,0.326051,0.000455
8,1,246,177.961,0.059,16.20910,0.015,177.961,0.014,0.009513,0.000062,0.430510,0.000486
9,1,246,963.265,0.270,9.15501,0.017,963.265,0.017,0.012839,0.000093,3.158987,0.001746


In [101]:
results_data = input_data_df

results_data.loc[:,'40Ar BC'] = results_data.loc[:, '40Ar'].values - (results_data.loc[:, '39Ar'].values * \
                                      irradiations_constants_df.loc['Interference 40K', 'value']) - background_spectrometer_df.loc['Background 40Ar', 'value']

results_data.loc[:,'1sigma_abs40'] = (pow(results_data.loc[:, 'err40Ar'].values, 2) + pow(background_spectrometer_df.loc['Background 40Ar', 'value'] * \
                                                                                            background_spectrometer_df.loc['Background 40Ar', 'relative error'], 2))**(1/2)

results_data.loc[:,'1sigma_rel40'] = results_data.loc[:,'1sigma_abs40'].values / results_data.loc[:,'40Ar BC'].values

results_data.loc[:,'39Ar BC'] = results_data.loc[:, '39Ar'].values - background_spectrometer_df.loc['Background 39Ar', 'value']

results_data.loc[:,'1sigma_abs39'] = (pow(results_data.loc[:, 'err39Ar'].values, 2) + pow(background_spectrometer_df.loc['Background 39Ar', 'value'] * \
                                                                                            background_spectrometer_df.loc['Background 39Ar', 'relative error'], 2))**(1/2)

results_data.loc[:,'1sigma_rel39'] = results_data.loc[:,'1sigma_abs39'].values / results_data.loc[:,'39Ar BC'].values

results_data.loc[:,'38Ar BC'] = results_data.loc[:, '38Ar'].values - background_spectrometer_df.loc['Background 38Ar', 'value']

results_data.loc[:,'1sigma_abs38'] = (pow(results_data.loc[:, 'err38Ar'].values, 2) + pow(background_spectrometer_df.loc['Background 38Ar', 'value'] * \
                                                                                            background_spectrometer_df.loc['Background 38Ar', 'relative error'], 2))**(1/2)

results_data.loc[:,'1sigma_rel38'] = results_data.loc[:,'1sigma_abs38'].values / results_data.loc[:,'38Ar BC'].values

results_data.loc[:,'37Ar BC'] = results_data.loc[:, '37Ar'].values - background_spectrometer_df.loc['Background 37Ar', 'value']

results_data.loc[:,'1sigma_abs37'] = (pow(results_data.loc[:, 'err37Ar'].values, 2) + pow(background_spectrometer_df.loc['Background 37Ar', 'value'] * \
                                                                                            background_spectrometer_df.loc['Background 37Ar', 'relative error'], 2))**(1/2)

results_data.loc[:,'1sigma_rel37'] = results_data.loc[:,'1sigma_abs37'].values / results_data.loc[:,'37Ar BC'].values

results_data.loc[:,'36Ar BC'] = results_data.loc[:, '36Ar'].values - background_spectrometer_df.loc['Background 36Ar', 'value']

results_data.loc[:,'1sigma_abs36'] = (pow(results_data.loc[:, 'err36Ar'].values, 2) + pow(background_spectrometer_df.loc['Background 36Ar', 'value'] * \
                                                                                            background_spectrometer_df.loc['Background 36Ar', 'relative error'], 2))**(1/2)

results_data.loc[:,'1sigma_rel36'] = results_data.loc[:,'1sigma_abs36'].values / results_data.loc[:,'36Ar BC'].values


display(results_data)

,Time,Delay,40Ar,err40Ar,39Ar,err39Ar,38Ar,err38Ar,37Ar,err37Ar,...,1sigma_rel39,38Ar BC,1sigma_abs38,1sigma_rel38,37Ar BC,1sigma_abs37,1sigma_rel37,36Ar BC,1sigma_abs36,1sigma_rel36
0,1,246,392.709,0.120,6.95503,0.014,392.709,0.014,0.008274,0.000076,...,0.002014,392.708,0.014,0.000036,0.004274,0.000804,0.188015,1.251026,0.000963,0.000770
1,1,246,244.726,0.076,16.39790,0.017,244.726,0.016,0.007669,0.000066,...,0.001037,244.725,0.016,0.000065,0.003669,0.000803,0.218802,0.645050,0.000730,0.001131
2,1,246,216.365,0.074,9.87657,0.016,216.365,0.017,0.007688,0.000071,...,0.001621,216.364,0.017,0.000079,0.003688,0.000803,0.217749,0.640230,0.000648,0.001013
3,1,246,225.166,0.085,10.35230,0.016,225.166,0.014,0.007593,0.000068,...,0.001546,225.165,0.014,0.000062,0.003593,0.000803,0.223463,0.666577,0.000610,0.000915
4,1,246,236.172,0.086,15.62530,0.019,236.172,0.017,0.007757,0.000074,...,0.001216,236.171,0.017,0.000072,0.003757,0.000803,0.213868,0.629161,0.000640,0.001017
5,1,246,277.142,0.093,1.61851,0.016,277.142,0.017,0.008214,0.000074,...,0.009906,277.141,0.017,0.000061,0.004214,0.000803,0.190679,0.919875,0.000946,0.001028
6,1,246,516.114,0.160,14.88850,0.014,516.114,0.014,0.009345,0.000072,...,0.000941,516.113,0.014,0.000027,0.005345,0.000803,0.150292,1.574012,0.001169,0.000743
7,1,246,134.762,0.038,12.27530,0.014,134.762,0.014,0.008279,0.000083,...,0.001141,134.761,0.014,0.000104,0.004279,0.000804,0.187977,0.325551,0.000458,0.001408
8,1,246,177.961,0.059,16.20910,0.015,177.961,0.014,0.009513,0.000062,...,0.000926,177.960,0.014,0.000079,0.005513,0.000802,0.145542,0.430010,0.000489,0.001136
9,1,246,963.265,0.270,9.15501,0.017,963.265,0.017,0.012839,0.000093,...,0.001858,963.264,0.017,0.000018,0.008839,0.000805,0.091122,3.158487,0.001747,0.000553


In [102]:
#results_data.loc[:,'Decay Factor'] = (irradiantions_df.loc['Lambda Ar37 [1/d]', 'value'] * \
#                                      results_data.loc[:, 'Time'].values * \
#                                        math.exp(irradiantions_df.loc['Lambda Ar37 [1/d]', 'value'] * \
 #                                       results_data.loc[:, 'Delay'].values))/(1-math.exp((-1) * \
 #                                       irradiantions_df.loc['Lambda Ar37 [1/d]', 'value'] * results_data.loc[:, 'Time'].values))

results_data.loc[:,'Decay Factor'] = (irradiations_constants_df.loc['Lambda Ar37 [1/d]', 'value'] * \
                                      results_data.loc[0, 'Time'] * \
                                        math.exp(irradiations_constants_df.loc['Lambda Ar37 [1/d]', 'value'] * \
                                        results_data.loc[0, 'Delay'])) / (1-math.exp((-1) * \
                                        irradiations_constants_df.loc['Lambda Ar37 [1/d]', 'value'] * 1))



display(results_data)

,Time,Delay,40Ar,err40Ar,39Ar,err39Ar,38Ar,err38Ar,37Ar,err37Ar,...,38Ar BC,1sigma_abs38,1sigma_rel38,37Ar BC,1sigma_abs37,1sigma_rel37,36Ar BC,1sigma_abs36,1sigma_rel36,Decay Factor
0,1,246,392.709,0.120,6.95503,0.014,392.709,0.014,0.008274,0.000076,...,392.708,0.014,0.000036,0.004274,0.000804,0.188015,1.251026,0.000963,0.000770,130.043076
1,1,246,244.726,0.076,16.39790,0.017,244.726,0.016,0.007669,0.000066,...,244.725,0.016,0.000065,0.003669,0.000803,0.218802,0.645050,0.000730,0.001131,130.043076
2,1,246,216.365,0.074,9.87657,0.016,216.365,0.017,0.007688,0.000071,...,216.364,0.017,0.000079,0.003688,0.000803,0.217749,0.640230,0.000648,0.001013,130.043076
3,1,246,225.166,0.085,10.35230,0.016,225.166,0.014,0.007593,0.000068,...,225.165,0.014,0.000062,0.003593,0.000803,0.223463,0.666577,0.000610,0.000915,130.043076
4,1,246,236.172,0.086,15.62530,0.019,236.172,0.017,0.007757,0.000074,...,236.171,0.017,0.000072,0.003757,0.000803,0.213868,0.629161,0.000640,0.001017,130.043076
5,1,246,277.142,0.093,1.61851,0.016,277.142,0.017,0.008214,0.000074,...,277.141,0.017,0.000061,0.004214,0.000803,0.190679,0.919875,0.000946,0.001028,130.043076
6,1,246,516.114,0.160,14.88850,0.014,516.114,0.014,0.009345,0.000072,...,516.113,0.014,0.000027,0.005345,0.000803,0.150292,1.574012,0.001169,0.000743,130.043076
7,1,246,134.762,0.038,12.27530,0.014,134.762,0.014,0.008279,0.000083,...,134.761,0.014,0.000104,0.004279,0.000804,0.187977,0.325551,0.000458,0.001408,130.043076
8,1,246,177.961,0.059,16.20910,0.015,177.961,0.014,0.009513,0.000062,...,177.960,0.014,0.000079,0.005513,0.000802,0.145542,0.430010,0.000489,0.001136,130.043076
9,1,246,963.265,0.270,9.15501,0.017,963.265,0.017,0.012839,0.000093,...,963.264,0.017,0.000018,0.008839,0.000805,0.091122,3.158487,0.001747,0.000553,130.043076


In [103]:
results_data.loc[:,'Mult 4amu'] = irradiations_df.loc['Gain corrected 40Ar/36Ar pipettes', 'relative error']
results_data.loc[:,'Mult 2amu'] = (results_data.loc[:,'Mult 4amu'].values + 1) / 2
results_data.loc[:,'Mult 1amu'] = (results_data.loc[:,'Mult 4amu'].values + 3) / 4

display(results_data)

,Time,Delay,40Ar,err40Ar,39Ar,err39Ar,38Ar,err38Ar,37Ar,err37Ar,...,37Ar BC,1sigma_abs37,1sigma_rel37,36Ar BC,1sigma_abs36,1sigma_rel36,Decay Factor,Mult 4amu,Mult 2amu,Mult 1amu
0,1,246,392.709,0.120,6.95503,0.014,392.709,0.014,0.008274,0.000076,...,0.004274,0.000804,0.188015,1.251026,0.000963,0.000770,130.043076,1.537691e-08,0.5,0.75
1,1,246,244.726,0.076,16.39790,0.017,244.726,0.016,0.007669,0.000066,...,0.003669,0.000803,0.218802,0.645050,0.000730,0.001131,130.043076,1.537691e-08,0.5,0.75
2,1,246,216.365,0.074,9.87657,0.016,216.365,0.017,0.007688,0.000071,...,0.003688,0.000803,0.217749,0.640230,0.000648,0.001013,130.043076,1.537691e-08,0.5,0.75
3,1,246,225.166,0.085,10.35230,0.016,225.166,0.014,0.007593,0.000068,...,0.003593,0.000803,0.223463,0.666577,0.000610,0.000915,130.043076,1.537691e-08,0.5,0.75
4,1,246,236.172,0.086,15.62530,0.019,236.172,0.017,0.007757,0.000074,...,0.003757,0.000803,0.213868,0.629161,0.000640,0.001017,130.043076,1.537691e-08,0.5,0.75
5,1,246,277.142,0.093,1.61851,0.016,277.142,0.017,0.008214,0.000074,...,0.004214,0.000803,0.190679,0.919875,0.000946,0.001028,130.043076,1.537691e-08,0.5,0.75
6,1,246,516.114,0.160,14.88850,0.014,516.114,0.014,0.009345,0.000072,...,0.005345,0.000803,0.150292,1.574012,0.001169,0.000743,130.043076,1.537691e-08,0.5,0.75
7,1,246,134.762,0.038,12.27530,0.014,134.762,0.014,0.008279,0.000083,...,0.004279,0.000804,0.187977,0.325551,0.000458,0.001408,130.043076,1.537691e-08,0.5,0.75
8,1,246,177.961,0.059,16.20910,0.015,177.961,0.014,0.009513,0.000062,...,0.005513,0.000802,0.145542,0.430010,0.000489,0.001136,130.043076,1.537691e-08,0.5,0.75
9,1,246,963.265,0.270,9.15501,0.017,963.265,0.017,0.012839,0.000093,...,0.008839,0.000805,0.091122,3.158487,0.001747,0.000553,130.043076,1.537691e-08,0.5,0.75


In [104]:
results_data.loc[:,'Ar36tot'] = results_data.loc[:,'36Ar BC'].values * results_data.loc[:,'Mult 4amu'].values
results_data.loc[:,'1sigRel36tot'] = (pow(results_data.loc[:,'1sigma_rel36'].values, 2) + \
                                      pow(irradiations_df.loc['Total fractionation uncertainty', 'value'], 2))**(1/2)

results_data.loc[:,'Ar38tot'] = results_data.loc[:,'38Ar BC'].values * results_data.loc[:,'Mult 2amu'].values
results_data.loc[:,'1sigRel38tot'] = (pow(results_data.loc[:,'1sigma_rel38'].values, 2) + 0.25 * \
                                      pow(irradiations_df.loc['Total fractionation uncertainty', 'value'], 2))**(1/2)

results_data.loc[:,'Ar39tot'] = results_data.loc[:,'39Ar BC'].values * results_data.loc[:,'Mult 1amu'].values
results_data.loc[:,'1sigRel39tot'] = (pow(results_data.loc[:,'1sigma_rel39'].values, 2) + 0.0625 * \
                                      pow(irradiations_df.loc['Total fractionation uncertainty', 'value'], 2))**(1/2)
                                                                        
display(results_data)

,Time,Delay,40Ar,err40Ar,39Ar,err39Ar,38Ar,err38Ar,37Ar,err37Ar,...,Decay Factor,Mult 4amu,Mult 2amu,Mult 1amu,Ar36tot,1sigRel36tot,Ar38tot,1sigRel38tot,Ar39tot,1sigRel39tot
0,1,246,392.709,0.120,6.95503,0.014,392.709,0.014,0.008274,0.000076,...,130.043076,1.537691e-08,0.5,0.75,1.923693e-08,0.003548,196.354003,0.001732,5.214023,0.002193
1,1,246,244.726,0.076,16.39790,0.017,244.726,0.016,0.007669,0.000066,...,130.043076,1.537691e-08,0.5,0.75,9.918881e-09,0.003644,122.362502,0.001733,12.296175,0.001351
2,1,246,216.365,0.074,9.87657,0.016,216.365,0.017,0.007688,0.000071,...,130.043076,1.537691e-08,0.5,0.75,9.844765e-09,0.003609,108.182002,0.001734,7.405178,0.001838
3,1,246,225.166,0.085,10.35230,0.016,225.166,0.014,0.007593,0.000068,...,130.043076,1.537691e-08,0.5,0.75,1.024990e-08,0.003583,112.582502,0.001733,7.761975,0.001772
4,1,246,236.172,0.086,15.62530,0.019,236.172,0.017,0.007757,0.000074,...,130.043076,1.537691e-08,0.5,0.75,9.674551e-09,0.003610,118.085502,0.001733,11.716725,0.001493
5,1,246,277.142,0.093,1.61851,0.016,277.142,0.017,0.008214,0.000074,...,130.043076,1.537691e-08,0.5,0.75,1.414484e-08,0.003613,138.570502,0.001733,1.211633,0.009944
6,1,246,516.114,0.160,14.88850,0.014,516.114,0.014,0.009345,0.000072,...,130.043076,1.537691e-08,0.5,0.75,2.420344e-08,0.003542,258.056504,0.001732,11.164125,0.001279
7,1,246,134.762,0.038,12.27530,0.014,134.762,0.014,0.008279,0.000083,...,130.043076,1.537691e-08,0.5,0.75,5.005963e-09,0.003739,67.380501,0.001735,9.204225,0.001432
8,1,246,177.961,0.059,16.20910,0.015,177.961,0.014,0.009513,0.000062,...,130.043076,1.537691e-08,0.5,0.75,6.612234e-09,0.003645,88.980001,0.001734,12.154575,0.001268
9,1,246,963.265,0.270,9.15501,0.017,963.265,0.017,0.012839,0.000093,...,130.043076,1.537691e-08,0.5,0.75,4.856779e-08,0.003508,481.632007,0.001732,6.864008,0.002050


In [105]:
 

results_data.loc[:,'Ar37day0'] = results_data.loc[:, 'Decay Factor'].values * results_data.loc[:,'37Ar BC'].values * \
                                 (results_data.loc[:,'Mult 4amu'].values * results_data.loc[:,'Mult 2amu'])
results_data.loc[:,'1sigRel37corr'] = (((pow(results_data.loc[:,'1sigma_rel37'].values, 2) + \
                                         pow(irradiations_df.loc['Total fractionation uncertainty', 'value'], 2)) * 9) / 16)**(1/2)
                                        
display(results_data)
                                                           

,Time,Delay,40Ar,err40Ar,39Ar,err39Ar,38Ar,err38Ar,37Ar,err37Ar,...,Mult 2amu,Mult 1amu,Ar36tot,1sigRel36tot,Ar38tot,1sigRel38tot,Ar39tot,1sigRel39tot,Ar37day0,1sigRel37corr
0,1,246,392.709,0.120,6.95503,0.014,392.709,0.014,0.008274,0.000076,...,0.5,0.75,1.923693e-08,0.003548,196.354003,0.001732,5.214023,0.002193,4.273264e-09,0.141035
1,1,246,244.726,0.076,16.39790,0.017,244.726,0.016,0.007669,0.000066,...,0.5,0.75,9.918881e-09,0.003644,122.362502,0.001733,12.296175,0.001351,3.667976e-09,0.164122
2,1,246,216.365,0.074,9.87657,0.016,216.365,0.017,0.007688,0.000071,...,0.5,0.75,9.844765e-09,0.003609,108.182002,0.001734,7.405178,0.001838,3.687571e-09,0.163333
3,1,246,225.166,0.085,10.35230,0.016,225.166,0.014,0.007593,0.000068,...,0.5,0.75,1.024990e-08,0.003583,112.582502,0.001733,7.761975,0.001772,3.592234e-09,0.167618
4,1,246,236.172,0.086,15.62530,0.019,236.172,0.017,0.007757,0.000074,...,0.5,0.75,9.674551e-09,0.003610,118.085502,0.001733,11.716725,0.001493,3.756050e-09,0.160422
5,1,246,277.142,0.093,1.61851,0.016,277.142,0.017,0.008214,0.000074,...,0.5,0.75,1.414484e-08,0.003613,138.570502,0.001733,1.211633,0.009944,4.212803e-09,0.143033
6,1,246,516.114,0.160,14.88850,0.014,516.114,0.014,0.009345,0.000072,...,0.5,0.75,2.420344e-08,0.003542,258.056504,0.001732,11.164125,0.001279,5.343818e-09,0.112749
7,1,246,134.762,0.038,12.27530,0.014,134.762,0.014,0.008279,0.000083,...,0.5,0.75,5.005963e-09,0.003739,67.380501,0.001735,9.204225,0.001432,4.277924e-09,0.141006
8,1,246,177.961,0.059,16.20910,0.015,177.961,0.014,0.009513,0.000062,...,0.5,0.75,6.612234e-09,0.003645,88.980001,0.001734,12.154575,0.001268,5.512493e-09,0.109188
9,1,246,963.265,0.270,9.15501,0.017,963.265,0.017,0.012839,0.000093,...,0.5,0.75,4.856779e-08,0.003508,481.632007,0.001732,6.864008,0.002050,8.837196e-09,0.068391


In [106]:
results_data.loc[:,'Ar39Ca'] = results_data.loc[:,'Ar37day0'].values * (irradiations_constants_df.loc['(39Ar/37Ar) Ca', 'value'])                                      
results_data.loc[:,'1sigRel39Ca'] = (pow(results_data.loc[:,'1sigRel37corr'].values, 2) + 0.000225)**(1/2)
results_data.loc[:,'1sigAbs39Ca'] = results_data.loc[:,'1sigRel39Ca'].values * results_data.loc[:,'Ar39Ca'].values  

results_data.loc[:,'Ar39K'] = results_data.loc[:,'Ar39tot'].values - results_data.loc[:,'Ar39Ca'].values
results_data.loc[:,'1sigAbs39K'] = (pow(results_data.loc[:,'1sigAbs39Ca'].values,2) + \
                                    pow(results_data.loc[:,'1sigRel39tot'].values * \
                                        results_data.loc[:,'Ar39tot'].values, 2))**(1/2)

results_data.loc[:,'Ar36Ca'] = results_data.loc[:,'Ar37day0'].values * irradiations_constants_df.loc['(36Ar/37Ar) Ca', 'value']  
results_data.loc[:,'1sigRel36Ca'] = (pow(results_data.loc[:,'1sigRel37corr'].values, 2) + 0.000225)**(1/2)                                                                     
results_data.loc[:,'1sigAbs36Ca'] = results_data.loc[:,'1sigRel36Ca'].values * results_data.loc[:,'Ar36Ca'].values
                                                                        
results_data.loc[:,'Ar36Atm'] = results_data.loc[:,'Ar36tot'].values - results_data.loc[:,'Ar36Ca'].values
results_data.loc[:,'1sigAbs36Atm'] = (pow(results_data.loc[:,'1sigAbs36Ca'].values,2) + \
                                      pow(results_data.loc[:,'1sigRel36tot'].values * \
                                          results_data.loc[:,'Ar36tot'].values, 2))**(1/2)  
results_data.loc[:,'1sigRel36Atm'] = results_data.loc[:,'1sigAbs36Atm'].values / results_data.loc[:,'Ar36Atm'].values
                                        
results_data.loc[:,'Ar40Atm'] = results_data.loc[:,'Ar36Atm'].values * irradiations_constants_df.loc['Atmospheric Ratio', 'value'] 
results_data.loc[:,'1sigAbs40Atm'] = results_data.loc[:,'Ar40Atm'].values * results_data.loc[:,'1sigRel36Atm'].values

results_data.loc[:, 'Ar40*'] = results_data.loc[:,'40Ar BC'].values - results_data.loc[:,'Ar40Atm'].values
results_data.loc[:,'1sigAbs40*'] = (pow(results_data.loc[:,'1sigma_abs40'].values, 2) + \
                                    pow(results_data.loc[:,'1sigAbs40Atm'].values, 2))**(1/2)

                                     
results_data.loc[:,'rendimento rad'] = results_data.loc[:, 'Ar40*'].values / results_data.loc[:,'40Ar BC'].values
results_data.loc[:,'error magnif'] = 1 - (results_data.loc[:,'rendimento rad'].values - 1)
results_data.loc[:,'error36*magnif'] = results_data.loc[:,'1sigRel36Atm'].values * results_data.loc[:,'error magnif'].values

                                     
results_data.loc[:,'1sigRel40*'] = results_data.loc[:,'1sigAbs40*'].values / results_data.loc[:, 'Ar40*'] 
results_data.loc[:,'Ar38Cl'] = (results_data.loc[:,'Ar38tot'].values - results_data.loc[:,'Ar39K'].values) / 90 - (results_data.loc[:,'Ar36Atm'].values * 0.18855) - (results_data.loc[:,'Ar37day0'].values * 0.00027)
results_data.loc[:,'1sigAbs38Cl'] = (pow(results_data.loc[:,'1sigRel38tot'].values, 2) + \
                                      pow((results_data.loc[:,'1sigAbs39K'].values / 85), 2) + \
                                       pow((results_data.loc[:,'1sigAbs36Atm'].values * 0.18855), 2) + \
                                        pow((results_data.loc[:,'1sigRel37corr'].values * results_data.loc[:,'Ar37day0'].values * 0.00027), 2))**(1/2)

results_data.loc[:,'1sigRel38Cl'] = results_data.loc[:,'1sigAbs38Cl'].values / results_data.loc[:,'Ar38Cl'].values


display(results_data)

,Time,Delay,40Ar,err40Ar,39Ar,err39Ar,38Ar,err38Ar,37Ar,err37Ar,...,1sigAbs40Atm,Ar40*,1sigAbs40*,rendimento rad,error magnif,error36*magnif,1sigRel40*,Ar38Cl,1sigAbs38Cl,1sigRel38Cl
0,1,246,392.709,0.120,6.95503,0.014,392.709,0.014,0.008274,0.000076,...,2.037882e-08,392.477478,0.120000,1.0,1.0,0.003548,0.000306,2.123778,0.001737,0.000818
1,1,246,244.726,0.076,16.39790,0.017,244.726,0.016,0.007669,0.000066,...,1.079082e-08,244.182866,0.076000,1.0,1.0,0.003644,0.000311,1.222959,0.001744,0.001426
2,1,246,216.365,0.074,9.87657,0.016,216.365,0.017,0.007688,0.000071,...,1.060718e-08,216.037070,0.074000,1.0,1.0,0.003609,0.000343,1.119742,0.001741,0.001555
3,1,246,225.166,0.085,10.35230,0.016,225.166,0.014,0.007593,0.000068,...,1.096352e-08,224.822371,0.085000,1.0,1.0,0.003583,0.000378,1.164673,0.001741,0.001494
4,1,246,236.172,0.086,15.62530,0.019,236.172,0.017,0.007757,0.000074,...,1.042720e-08,235.654362,0.086000,1.0,1.0,0.003610,0.000365,1.181875,0.001746,0.001477
5,1,246,277.142,0.093,1.61851,0.016,277.142,0.017,0.008214,0.000074,...,1.525894e-08,277.086585,0.093000,1.0,1.0,0.003613,0.000336,1.526210,0.001739,0.001139
6,1,246,516.114,0.160,14.88850,0.014,516.114,0.014,0.009345,0.000072,...,2.559862e-08,515.620672,0.160000,1.0,1.0,0.003543,0.000310,2.743249,0.001740,0.000634
7,1,246,134.762,0.038,12.27530,0.014,134.762,0.014,0.008279,0.000083,...,5.588147e-09,134.354914,0.038001,1.0,1.0,0.003740,0.000283,0.646403,0.001742,0.002695
8,1,246,177.961,0.059,16.20910,0.015,177.961,0.014,0.009513,0.000062,...,7.196572e-09,177.424098,0.059000,1.0,1.0,0.003646,0.000333,0.853616,0.001743,0.002042
9,1,246,963.265,0.270,9.15501,0.017,963.265,0.017,0.012839,0.000093,...,5.086188e-08,962.960870,0.270000,1.0,1.0,0.003508,0.000280,5.275200,0.001740,0.000330


In [107]:
#colonne RESULTS

results_data.loc[:,'40Ar_total'] = irradiations_df.loc['Sensitivity (mL/mV)', 'value'] * results_data.loc[:,'40Ar BC'].values
results_data.loc[:,'err_40Ar'] = results_data.loc[:,'40Ar_total'].values * results_data.loc[:,'1sigma_rel40'].values

results_data.loc[:,'40Ar*'] = irradiations_df.loc['Sensitivity (mL/mV)', 'value'] * results_data.loc[:, 'Ar40*'].values
results_data.loc[:, 'err_40Ar*'] = results_data.loc[:,'1sigRel40*'].values * results_data.loc[:,'40Ar*'].values

results_data.loc[:,'39Ar'] = irradiations_df.loc['Sensitivity (mL/mV)', 'value'] * results_data.loc[:,'Ar39tot'].values
results_data.loc[:,'err_39Ar'] = results_data.loc[:,'39Ar'] * results_data.loc[:,'1sigRel39tot'].values

# inserire variabile cella BP3 = sommatoria colonne 39Ar (BP)
total_mL_39Ar = results_data.loc[:,'39Ar'].sum()

results_data.loc[:,'% 39Ar'] = 100 * (results_data.loc[:,'39Ar'].values / total_mL_39Ar)                                                         

results_data.loc[:,'38Ar'] = irradiations_df.loc['Sensitivity (mL/mV)', 'value'] * results_data.loc[:,'Ar38tot'].values                                                                                           
results_data.loc[:,'err_38Ar'] = results_data.loc[:,'38Ar'].values * results_data.loc[:,'1sigRel38tot'].values

results_data.loc[:,'38Ar_Cl'] = irradiations_df.loc['Sensitivity (mL/mV)', 'value'] * results_data.loc[:,'Ar38Cl'].values
results_data.loc[:,'err_38Cl'] = results_data.loc[:,'1sigRel38Cl'].values * results_data.loc[:,'38Ar_Cl'].values
                                 
results_data.loc[:,'37Ar'] = irradiations_df.loc['Sensitivity (mL/mV)', 'value'] * results_data.loc[:,'Ar37day0'].values
results_data.loc[:,'err_37Ar'] = results_data.loc[:,'1sigRel37corr'].values *  results_data.loc[:,'37Ar'].values

results_data.loc[:,'36Ar'] = irradiations_df.loc['Sensitivity (mL/mV)', 'value'] * results_data.loc[:,'Ar36tot'].values
results_data.loc[:,'err_36Ar'] = results_data.loc[:,'1sigRel36tot'].values * results_data.loc[:,'36Ar'].values

results_data.loc[:,'Age'] = np.log(1 + (results_data.loc[:, '40Ar*'].values * \
                                        irradiations_df.loc['J factor', 'value'] / \
                                        results_data.loc[:,'Ar39K'].values) / \
                                        irradiations_constants_df.loc['Lambda Ar40 [1/Ma]', 'value'])


results_data.loc[:,'1sigma_err_Age'] = results_data.loc[:,'Age'].values * \
                                        (pow(irradiations_df.loc['J factor', 'relative error'], 2) * \
                                         pow(results_data.loc[:,'1sigRel40*'].values, 2) * \
                                         pow(results_data.loc[:,'1sigAbs39K'].values / \
                                             results_data.loc[:,'Ar39K'].values, 2))**(1/2)

results_data.loc[:,'Age+2error'] = results_data.loc[:,'Age'].values + 2 * results_data.loc[:,'1sigma_err_Age'].values
results_data.loc[:,'Age-2error'] = results_data.loc[:,'Age'].values - 2 * results_data.loc[:,'1sigma_err_Age'].values    

results_data.loc[:,'Ca/K'] =  results_data.loc[:,'Ar37day0'].values * (1.94 / results_data.loc[:,'Ar39K'].values)   
results_data.loc[:,'err_Ca/K'] = results_data.loc[:,'Ca/K'].values * \
                                    (pow(results_data.loc[:,'1sigRel37corr'].values, 2) + \
                                     pow(results_data.loc[:,'1sigAbs39K'].values / results_data.loc[:,'Ar39K'].values, 2))**(1/2)

results_data.loc[:,'Cl/K'] = irradiations_constants_df.loc['Coefficient Cl/K', 'value'] * (results_data.loc[:,'38Ar_Cl'].values * results_data.loc[:,'39Ar'].values)
results_data.loc[:,'err_Cl/K'] = results_data.loc[:,'Cl/K'].values * \
                                    (pow(results_data.loc[:,'1sigRel38Cl'].values, 2) + \
                                     pow(results_data.loc[:,'1sigAbs39K'].values / \
                                         results_data.loc[:,'Ar39K'].values, 2))**(1/2)

results_data.loc[:,'39/40'] = results_data.loc[:,'Ar39K'].values / results_data.loc[:,'40Ar BC'].values
results_data.loc[:,'Sig_9'] = results_data.loc[:,'39/40'] * \
                                (pow(results_data.loc[:,'1sigma_rel40'].values, 2) + \
                                 pow(results_data.loc[:,'1sigAbs39K'].values / \
                                     results_data.loc[:,'Ar39K'].values, 2))**(1/2)

results_data.loc[:,'36/40'] = (results_data.loc[:,'Ar36tot'].values - (results_data.loc[:,'Ar37day0'].values * \
                                irradiations_constants_df.loc['(36Ar/37Ar) Ca', 'value'])) / results_data.loc[:,'40Ar BC'].values
results_data.loc[:,'Sig_6'] = results_data.loc[:,'36/40'].values * (pow(results_data.loc[:,'1sigma_rel40'].values, 2) + \
                                                                    pow(results_data.loc[:,'1sigRel36Atm'].values, 2))**(1/2)

display(results_data)

,Time,Delay,40Ar,err40Ar,39Ar,err39Ar,38Ar,err38Ar,37Ar,err37Ar,...,Age+2error,Age-2error,Ca/K,err_Ca/K,Cl/K,err_Cl/K,39/40,Sig_9,36/40,Sig_6
0,1,246,392.709,0.120,1.757126e-10,0.014,6.617130e-09,0.014,1.440090e-19,0.000076,...,5.227199e-09,5.227199e-09,1.589969e-09,2.242690e-10,2.238524e-21,5.238520e-24,0.013285,0.000029,4.901131e-11,1.745577e-13
1,1,246,244.726,0.076,4.143811e-10,0.017,4.123616e-09,0.016,1.236108e-19,0.000066,...,1.379026e-09,1.379026e-09,5.787063e-10,9.498186e-11,3.039918e-21,5.971867e-24,0.050356,0.000070,4.061688e-11,1.485546e-13
2,1,246,216.365,0.074,2.495545e-10,0.016,3.645733e-09,0.017,1.242711e-19,0.000071,...,2.025909e-09,2.025909e-09,9.660657e-10,1.578001e-10,1.676229e-21,4.034914e-24,0.034277,0.000064,4.556544e-11,1.651914e-13
3,1,246,225.166,0.085,2.615786e-10,0.016,3.794030e-09,0.014,1.210583e-19,0.000068,...,2.011382e-09,2.011382e-09,8.978299e-10,1.505005e-10,1.827494e-21,4.236531e-24,0.034525,0.000063,4.558702e-11,1.642416e-13
4,1,246,236.172,0.086,3.948536e-10,0.019,3.979481e-09,0.017,1.265789e-19,0.000074,...,1.396678e-09,1.396678e-09,6.219090e-10,9.977234e-11,2.799353e-21,5.879110e-24,0.049720,0.000076,4.104992e-11,1.489594e-13
5,1,246,277.142,0.093,4.083202e-11,0.016,4.669826e-09,0.017,1.419715e-19,0.000074,...,1.588078e-08,1.588078e-08,6.745310e-09,9.671286e-10,3.738221e-22,3.741421e-24,0.004373,0.000044,5.104458e-11,1.852434e-13
6,1,246,516.114,0.160,3.762310e-10,0.014,8.696504e-09,0.014,1.800867e-19,0.000072,...,3.207250e-09,3.207250e-09,9.286000e-10,1.047056e-10,6.191126e-21,8.836693e-24,0.021652,0.000028,4.693776e-11,1.669223e-13
7,1,246,134.762,0.038,3.101824e-10,0.014,2.270723e-09,0.014,1.441661e-19,0.000083,...,1.013663e-09,1.013663e-09,9.016700e-10,1.271478e-10,1.202736e-21,3.670535e-24,0.068507,0.000100,3.725113e-11,1.397079e-13
8,1,246,177.961,0.059,4.096092e-10,0.015,2.998626e-09,0.014,1.857710e-19,0.000062,...,1.013678e-09,1.013678e-09,8.798527e-10,9.607549e-11,2.097404e-21,5.041095e-24,0.068506,0.000090,3.726004e-11,1.364207e-13
9,1,246,963.265,0.270,2.313171e-10,0.017,1.623100e-08,0.017,2.978135e-19,0.000093,...,9.742223e-09,9.742223e-09,2.497690e-09,1.708962e-10,7.319754e-21,1.519632e-23,0.007128,0.000015,5.043355e-11,1.774742e-13


In [108]:
# for spectrum 
results_data.loc[:,'cum_%39'] = 
results_data.loc[:,'Age+'] = 
results_data.loc[:,'Age+'] =   

SyntaxError: invalid syntax (<ipython-input-108-247d53860616>, line 2)